In [1]:
import requests
import pandas as pd
import gspread
import df2gspread as d2g
import re

In [2]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    tagless_text = TAG_RE.sub(' ', text)
    spaced_text = re.sub(' +', ' ', tagless_text)
    return spaced_text
    

In [3]:
def split_text(text, split_on = ['Characteristics \n','\n Details \n', '\n Roasting \n']):
    return re.split('Characteristics \n|\n Details \n|\n Roasting \n',text)

In [4]:
test = split_text("test Characteristics \n test1 \n Details \n test2 \n Roasting \n' test3")

In [5]:
print(test)

['test ', ' test1 ', ' test2 ', "' test3"]


In [6]:
class ShopifyScraper():
    def __init__(self,root_domain):
        self.domain_url = root_domain
        self.product_list_url = self.domain_url + '/products.json'
        self.product_dict = {}
        
    def get_products(self):
        self.fetch_products = requests.get(self.product_list_url)
        products = self.fetch_products.json()
        #for product in products['products']:
        #    print(product)
        #    print()
       
        for product in products['products']:
            for variant in product['variants']:
                
                #print(variant['title'])

                product_dict_detailed = {}
                product_dict_detailed['product'] = product['title']
                product_dict_detailed['slug'] = product['handle']
                product_dict_detailed['product_type'] = product['product_type']
                product_dict_detailed['beans/nibs'] = variant['option1']
                product_dict_detailed['raw/roasted'] = variant['option2']
                product_dict_detailed['size (lbs)'] = variant['option3']
                product_dict_detailed['size (grams)'] = variant['grams']
                product_dict_detailed['beans/nibs'] = variant['option1']
                product_dict_detailed['available'] = variant['available']
                product_dict_detailed['price'] = variant['price']
                product_dict_detailed['product_id'] = variant['product_id']
                product_dict_detailed['update_date'] = variant['updated_at']
                
                
                
                tagless = remove_tags(product['body_html'])
                '''
                product_dict_detailed['characteristics'] = tagless
                
                
                '''
                splits = split_text(tagless)
                #print(splits)
                try:
                    product_dict_detailed['characteristics'] = splits[1]
                    product_dict_detailed['details'] = splits[2]
                    product_dict_detailed['roasting'] = splits[3]
                except:
                    product_dict_detailed['characteristics'] = tagless
                    product_dict_detailed['details'] = tagless
                    product_dict_detailed['roasting'] = tagless
                #'''
                
                self.product_dict[f"{product['title']} / {variant['title']}"] = product_dict_detailed

    def print_products(self):
        for product in self.product_dict:
            print(product)

In [7]:
x = ShopifyScraper("https://chocolatealchemy2.myshopify.com")
x.get_products()
#x.print_products()

In [8]:
x.product_dict

{'Tanzania - Direct trade/Organic - 2019 / Whole Beans / Raw / 10 lb': {'product': 'Tanzania - Direct trade/Organic - 2019',
  'slug': 'tanzania-direct-trade-2019',
  'product_type': 'Beans',
  'beans/nibs': 'Whole Beans',
  'raw/roasted': 'Raw',
  'size (lbs)': '10 lb',
  'size (grams)': 4876,
  'available': True,
  'price': '81.70',
  'product_id': 3766090884,
  'update_date': '2021-05-07T08:32:10-07:00',
  'characteristics': 'Origin: Tanzania - Type: Criollo/Trinatario Certifications: Direct Trade Kokoa Kamili Cooperative, Organic Harvest Year: 2019 Flavor Notes: There is a clean earthiness and up front chocolate aroma. There are deep base fruits, dried prune and damson plum in the nose. The chocolate has dark flavors of brown sugar, toffee, coffee and a fully balanced citrus (lemon?) acidity. Soft cashew, raisin and a silkiness are the overall impression. There is a lingering high note in the finish of bittering spice and baked raisin. In many ways it puts me in mind of an English 

In [9]:
ChocolateAlchemy_Products = pd.DataFrame(x.product_dict)
ChocolateAlchemy_Products = ChocolateAlchemy_Products.transpose()
ChocolateAlchemy_Beans = ChocolateAlchemy_Products[(ChocolateAlchemy_Products['product_type'] == 'Beans') & (ChocolateAlchemy_Products['available'] == True)]

In [10]:
ChocolateAlchemy_Beans

,product,slug,product_type,beans/nibs,raw/roasted,size (lbs),size (grams),available,price,product_id,update_date,characteristics,details,roasting
Tanzania - Direct trade/Organic - 2019 / Whole Beans / Raw / 10 lb,Tanzania - Direct trade/Organic - 2019,tanzania-direct-trade-2019,Beans,Whole Beans,Raw,10 lb,4876,True,81.70,3766090884,2021-05-07T08:32:10-07:00,Origin: Tanzania - Type: Criollo/Trinatario Ce...,Kokoa Kamili currently works with approximatel...,"If you oven roast, I suggest the following. P..."
Tanzania - Direct trade/Organic - 2019 / Whole Beans / Raw / 25 lb,Tanzania - Direct trade/Organic - 2019,tanzania-direct-trade-2019,Beans,Whole Beans,Raw,25 lb,12247,True,172.00,3766090884,2021-03-16T14:53:15-07:00,Origin: Tanzania - Type: Criollo/Trinatario Ce...,Kokoa Kamili currently works with approximatel...,"If you oven roast, I suggest the following. P..."
Tanzania - Direct trade/Organic - 2019 / Whole Beans / Raw / 1 bag (65 kg) $5/lb ex-warehouse West Coast,Tanzania - Direct trade/Organic - 2019,tanzania-direct-trade-2019,Beans,Whole Beans,Raw,1 bag (65 kg) $5/lb ex-warehouse West Coast,65771,True,715.00,3766090884,2021-04-08T11:04:55-07:00,Origin: Tanzania - Type: Criollo/Trinatario Ce...,Kokoa Kamili currently works with approximatel...,"If you oven roast, I suggest the following. P..."
Tanzania - Direct trade/Organic - 2019 / Whole Beans / Raw / 1/2 MT $3.25/lb ex-warehouse West Coast,Tanzania - Direct trade/Organic - 2019,tanzania-direct-trade-2019,Beans,Whole Beans,Raw,1/2 MT $3.25/lb ex-warehouse West Coast,499999,True,3572.00,3766090884,2021-04-08T11:06:36-07:00,Origin: Tanzania - Type: Criollo/Trinatario Ce...,Kokoa Kamili currently works with approximatel...,"If you oven roast, I suggest the following. P..."
Tanzania - Direct trade/Organic - 2019 / Whole Beans / Raw / 1 MT $3.00/lb ex-warehouse West Coast,Tanzania - Direct trade/Organic - 2019,tanzania-direct-trade-2019,Beans,Whole Beans,Raw,1 MT $3.00/lb ex-warehouse West Coast,999999,True,6600.00,3766090884,2021-04-08T11:06:36-07:00,Origin: Tanzania - Type: Criollo/Trinatario Ce...,Kokoa Kamili currently works with approximatel...,"If you oven roast, I suggest the following. P..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ivory Coast - RFA - 2016 / Nibs / Raw / 10 lb,Ivory Coast - RFA - 2016,ivory-coast-rfa-2016,Beans,Nibs,Raw,10 lb,4876,True,77.19,426088200,2021-03-19T07:39:41-07:00,Origin: Ivory Coast Crop: 2016 Type: Forastero...,"As for the cocoa itself, there is a second com...",This is also a good beginner bean. I like this...
Ivory Coast - RFA - 2016 / Nibs / Raw / 25 lb,Ivory Coast - RFA - 2016,ivory-coast-rfa-2016,Beans,Nibs,Raw,25 lb,12247,True,162.50,426088200,2021-03-19T07:39:41-07:00,Origin: Ivory Coast Crop: 2016 Type: Forastero...,"As for the cocoa itself, there is a second com...",This is also a good beginner bean. I like this...
Ivory Coast - RFA - 2016 / Whole Beans / Roasted / 1 bag (65 kg),Ivory Coast - RFA - 2016,ivory-coast-rfa-2016,Beans,Whole Beans,Roasted,1 bag (65 kg),65771,True,669.47,426088200,2021-03-19T07:39:40-07:00,Origin: Ivory Coast Crop: 2016 Type: Forastero...,"As for the cocoa itself, there is a second com...",This is also a good beginner bean. I like this...
Ivory Coast - RFA - 2016 / Nibs / Roasted / 1 bag (65 kg) $5.95/lb ex-warehouse West Coast,Ivory Coast - RFA - 2016,ivory-coast-rfa-2016,Beans,Nibs,Roasted,1 bag (65 kg) $5.95/lb ex-warehouse West Coast,65771,True,776.95,426088200,2021-03-19T07:39:40-07:00,Origin: Ivory Coast Crop: 2016 Type: Forastero...,"As for the cocoa itself, there is a second com...",This is also a good beginner bean. I like this...


In [11]:
gc = gspread.oauth()

In [12]:
cacow_sheet = gc.open('Chocolateering')

In [13]:
cacow_sheet.values_clear("'ChocolateAlchemy_beans'!A1:Z500")

{'spreadsheetId': '1GD0CVYTGsd_FEC7GSVhqpDVhZGOXycC8KJK5uDcY6Mg',
 'clearedRange': 'ChocolateAlchemy_beans!A1:Z500'}

In [14]:
cacow_sheet_CA_products = cacow_sheet.worksheet('ChocolateAlchemy_beans')

In [15]:
cacow_sheet_CA_products.update([ChocolateAlchemy_Beans.columns.values.tolist()] + ChocolateAlchemy_Beans.values.tolist())

{'spreadsheetId': '1GD0CVYTGsd_FEC7GSVhqpDVhZGOXycC8KJK5uDcY6Mg',
 'updatedRange': 'ChocolateAlchemy_beans!A1:N234',
 'updatedRows': 234,
 'updatedColumns': 14,
 'updatedCells': 3276}